# Técnicas de redução de dimensionalidade

Na era do Big Data, os dados não só estão se tornando cada vez maiores como também mais complexos. Isso se traduz em um aumento espetacular da dimensionalidade dos dados. Por exemplo, a dimensionalidade de um conjunto de imagens é o número de pixels que ela contém, que pode variar de milhares a milhões.

Computacionalmente não existe problema em processar tantas dimensões, no entanto, nós humanos somos limitados a três dimensões. Dessa forma, precisamos de maneiras para visualizar dados com alta dimensionalidade para avalia-los antes de usar como entrada para algum outro processo/algorítmo, como Machine Learning por exemplo.

### Como podemos reduzir a dimensionalidade de um dataset de um número arbitrário para dois ou três, que é o que costumamos fazer para visualizar dados em uma tela?


## Analogia : Fotografia

A resposta pode ser melhor entendida com uma implementação real de redução de dimensionalidade : a **câmera fotográfica**

<img src='photo_example.jpg' height="382" width="574">

A camera fotográfica, por meio da fotografia, representa uma projeção de um espaço multidimensional (mundo real) em um plano (foto). As características originais do espaço de alta dimensionalidade são mantidos na nova projeção com baixa dimensionalidade.

### MNIST dataset

Para esse exercício utilizaremos o dataset MNIST (http://yann.lecun.com/exdb/mnist/) que consiste em 70.000 imagens de dígitos de 0 a 9 escritos a mão.

Não precisamos realizar o download do dataset manualmente, podemos baixá-lo através do Scikit learn, utilizando o comando ***fetch_mldata("MNIST original")***.

In [ ]:
#import das bibliotecas básicas utilizadas neste trabalho
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata("MNIST original")
X = mnist.data / 255.0
y = mnist.target

print(X.shape, y.shape)

Para facilitar a manipulação dos dados, vamos converter a matriz original em um DataFrame do Pandas.

In [ ]:
feat_cols = [ 'pixel'+str(i) for i in range(X.shape[1]) ]

df = pd.DataFrame(X,columns=feat_cols)
df['label'] = y
df['label'] = df['label'].apply(lambda i: str(i))

X, y = None, None

print('Tamanho do dataframe: {}'.format(df.shape))


Já que não queremos utilizar todos os 70.000 dígitos do dataset em alguns cálculos, nós vamos extrair uma amostra aleatória dos dados. A "randomização" é importante pois o dataset original está ordenado por rótulo (ex.: as primeiras 7 mil imagens são zeros, e assim por diante). Para garantir essa aleatoriedade, vamos utilizar uma função do numpy chamada ***np.random.permutation***, utilizando um número entre 0 e 69.999.

In [3]:
rndperm = np.random.permutation(df.shape[0])

Agora temos um DataFrame e um vetor com índices randomizados. Primeiro, vamos verificar como os dados do dataset se parecem, selecionando 30 imagens aleatórias.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plota o gráfico
plt.gray()
fig = plt.figure( figsize=(16,7) )
for i in range(0,30):
    ax = fig.add_subplot(3,10,i+1, title='Digit: ' + str(df.loc[rndperm[i],'label']) )
    ax.matshow(df.loc[rndperm[i],feat_cols].values.reshape((28,28)).astype(float))
plt.show()


As imagens estão todas no formato 28x28 píxels, portanto temos um total de 784 dimensões (28x28=784), cada uma contendo o valor de um píxel específico.

O que podemos fazer é reduzir o número de dimensões drasticamente enquanto tentamos reter a maior 'variação' da informação possível. Isto é o que chamamos de **redução de dimensionalidade**. 

Primeiro, vamos utilizar um conceito mais simples, conhecido como Principal Component Analysis (**PCA**).

## Redução de dimensionalidade utilizando PCA

PCA é uma técnica para redução do número de dimensões em um dataset retendo maior parte da informação. Ela usa a correlação entre algumas dimensões e tenta prover um número mínimo de variáveis que mantém a maior quantidade de variação da informação sobre como os dados estão originalmente distribuídos. 

Não vamos descer ao detalhe de como são calculados os componentes principais do PCA, mas esse link [aqui](https://www.math.hmc.edu/calculus/tutorials/eigenstuff/) aborda como eles são calculados. 

Já que nós humanos gostamos de plots 2/3D, vamos começar gerando os três primeiros componentes principais a partir das 784 dimensões originais. Nós vamos ver quanto da variação total do dataset cada componente é responsável.

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(df[feat_cols].values)

df['pca-um'] = pca_result[:,0]
df['pca-dois'] = pca_result[:,1] 
df['pca-tres'] = pca_result[:,2]

print('Variação explicada por cada componente : {}'.format(pca.explained_variance_ratio_))

Agora, dado que os dois primeiros componentes são responsáveis por cerca de 17% da variação em todo o dataset, vamos ver se isso é suficiente para separar cada dígito visualmente. O que faremos é criar um scatterplot do primeiro e segundo componente principal e colorir cada um dos diferentes dígitos com uma cor diferente. Se tivermos sorte os dígitos estarão posicionados juntos (clusterizados), o que significaria que os dois principais componentes realmente representam uma boa projeção dos dígitos.

In [ ]:
# Cria a figura
fig = plt.figure( figsize=(8,8) )
ax = fig.add_subplot(1, 1, 1, title='PCA' )
# Cria o scatterplot
ax.scatter(
    x=df['pca-um'], 
    y=df['pca-dois'], 
    c=df['label'], 
    cmap=plt.cm.get_cmap('Paired'), 
    alpha=0.15)

A partir do gráfico nós podemos ver que os dois componentes definitivamente contém alguma informação, especialmente para alguns dígitos, mas claramente não são suficientes para separar todo o dataset. Abaixo veremos um pouco sobre uma outra técnica e exploraremos se ela pode nos dar uma melhor maneira de reduzir a dimensionalidade para visualização. O método que vamos explorar é conhecido como t-SNE (t-Distributed Stochastic Neighbouring Entities).

### t-SNE

Esse material é uma introdução a um algorítmo popular de redução de dimensionalidade : t-distributed stochastic neighbor embedding (**t-SNE**). Desenvolvido por Laurens van der Maaten e Geoffrey Hinton ([link para o artigo original](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf)), este algorítmo tem sido aplicado com sucesso em vários conjuntos de dados. Aqui, vamos focar na aplicação da técnica utilizando Python e a biblioteca scikit-learn.

In [ ]:
import time

from sklearn.manifold import TSNE

#para efeitos de demonstração limitaremos os dados a uma amostra de n_sne valores aleatórios.
n_sne = 7000

time_start = time.time()
tsne = TSNE(random_state=42, verbose=1)
tsne_results = tsne.fit_transform(df.loc[rndperm[:n_sne],feat_cols].values)

print('t-SNE pronto! Tempo decorrido: {} segundos'.format(time.time()-time_start))

In [8]:
df_tsne = df.loc[rndperm[:n_sne],:].copy()
df_tsne['x-tsne'] = tsne_results[:,0]
df_tsne['y-tsne'] = tsne_results[:,1]

Agora que temos as duas dimensões resultantes do processo de redução de dimensionalidade pelo t-SNE, podemos novamente visualizar o resultado através de um scatterplot com as duas dimensões nos eixos x e y, e colorindo cada dígito de acordo com seu label.

In [ ]:
# Cria a figura
fig = plt.figure( figsize=(8,8) )
ax = fig.add_subplot(1, 1, 1, title='tSNE' )
# Cria o scatterplot
ax.scatter(
    x=df_tsne['x-tsne'], 
    y=df_tsne['y-tsne'], 
    c=df_tsne['label'], 
    cmap=plt.cm.get_cmap('Paired'), 
    alpha=0.25)


O resultado trás uma melhoria significativa sobre o PCA utilizado anteriormente. Podemos ver que os dígitos estão claramente agrupados no seu próprio grupo.

Porém, vamos trabalhar o gráfico para melhor visualizar cada cluster e seu label correspondente.

In [12]:
# Seaborn para incrementar um pouco os gráficos :D
import seaborn as sns
import matplotlib.patheffects as PathEffects
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

def scatter(x, y):
    palette = np.array(sns.color_palette("hls", 10))

    # Criamos o scatterplot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[y.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # Adicionamos o label de cada dígito no centro de cada cluster
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[y == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
scatter(tsne_results, pd.to_numeric(df.loc[rndperm[:n_sne],'label']).values.astype(int))
plt.show()

Claramente vemos que alguns dígitos estão com sobreposição.

Existe uma recomendação que diz que, em caso de dados com dimensionalidade muito alta, devemos aplicar uma outra técnica de redução de dimensionalidade antes de usar o t-SNE.

Seguindo essa recomendação, vamos usar PCA novamente. Primeiro criaremos um novo dataset contendo 50 dimensões geradas pela redução via PCA.

In [ ]:
#TODO : gerar PCA para os 50 primeiros principais componentes
pca_50 = None
pca_result_50 = None

print('Variação explicada pelos 50 componentes principais: {}'.format(np.sum(pca_50.explained_variance_ratio_)))

Wow! Os primeiros 50 componentes explicam cerca de 82% da variação total nos dados.

Agora, vamos tentar utilizar esses componentes como entrada para o t-SNE. Desta vez utilizaremos uma amostra de 10.000 para não sobrecarregar demais a memória e o processador.

In [ ]:
n_sne = 10000

time_start = time.time()

#TODO : Executar o t-SNE sobre as 50 features extraídas pelo PCA
tsne = None
tsne_pca_results = None

print('t-SNE pronto! Tempo decorrido: {} segundos'.format(time.time()-time_start))

In [ ]:
scatter(tsne_pca_results, pd.to_numeric(df.loc[rndperm[:n_sne],'label']).values.astype(int))
plt.show()

### Conclusão
Após seguir a recomendação e realizar primeiro a redução de dimensionalidade via PCA, reduzindo para 50 features, e depois via t-SNE, quais foram as melhorias percebidas?

*descreva a diferença entre o plot anterior gerado utilizando as 784 features e o plot acima.*

**Resposta :**